### 07_IMD_Neighbour_Methods

Test multiple approaches for deriving missing IMD scores using neighbouring LSOAs.


In [ ]:
import os
import pandas as pd
import geopandas as gpd
from sklearn.neighbors import NearestNeighbors
import numpy as np

RAW_DATA_DIR = '../data/raw'
PROCESSED_DATA_DIR = '../data/processed'
LSOA_SW_PATH = os.path.join(PROCESSED_DATA_DIR, 'lsoa_sw_enriched_demographics.gpkg')


In [ ]:
# Load enriched LSOA data
lsoa_sw = gpd.read_file(LSOA_SW_PATH, layer='lsoa_sw_enriched_demographics')
lsoa_sw['centroid'] = lsoa_sw.geometry.centroid


In [ ]:
def impute_imd_knn_mean(gdf, imd_col='IMD_Rank', k=5):
    coords = np.array([(geom.x, geom.y) for geom in gdf['centroid']])
    missing = gdf[imd_col].isna()
    nn = NearestNeighbors(n_neighbors=k).fit(coords[~missing])
    d, idx = nn.kneighbors(coords[missing])
    gdf.loc[missing, imd_col] = gdf.loc[~missing, imd_col].values[idx].mean(axis=1)
    return gdf


In [ ]:
def impute_imd_knn_median(gdf, imd_col='IMD_Rank', k=5):
    coords = np.array([(geom.x, geom.y) for geom in gdf['centroid']])
    missing = gdf[imd_col].isna()
    nn = NearestNeighbors(n_neighbors=k).fit(coords[~missing])
    d, idx = nn.kneighbors(coords[missing])
    gdf.loc[missing, imd_col] = np.median(gdf.loc[~missing, imd_col].values[idx], axis=1)
    return gdf


In [ ]:
def impute_imd_knn_distance_weighted(gdf, imd_col='IMD_Rank', k=5, epsilon=1e-8):
    coords = np.array([(geom.x, geom.y) for geom in gdf['centroid']])
    missing = gdf[imd_col].isna()
    nn = NearestNeighbors(n_neighbors=k).fit(coords[~missing])
    d, idx = nn.kneighbors(coords[missing])
    weights = 1.0 / (d + epsilon)
    values = gdf.loc[~missing, imd_col].values[idx]
    gdf.loc[missing, imd_col] = np.sum(values * weights, axis=1) / np.sum(weights, axis=1)
    return gdf


In [ ]:
# Example usage (choose one method)
# lsoa_sw = impute_imd_knn_mean(lsoa_sw.copy())
# lsoa_sw = impute_imd_knn_median(lsoa_sw.copy())
# lsoa_sw = impute_imd_knn_distance_weighted(lsoa_sw.copy())
